In [2]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()

In [3]:
# result = []
# json = []
# for row in adata.obs.values:
#     result.append([item for kev, item in enumerate(row) ])
# for row in result: json.append({k: v for k, v in zip(adata.obs.columns.tolist(), row)})

# print(json)

In [4]:

sc.pp.highly_variable_genes(adata)

NameError: name 'result' is not defined

In [5]:
#preprocess
#batchcontrol


In [6]:
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation

#write custom model. get better accuracy

In [7]:
sc.pp.calculate_qc_metrics(adata)
5

5

In [8]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)
    for result in results:
        print('url:', result[0], 'time (s):', result[1])

In [9]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [10]:
gene_expressed = ["string1", "string2", "string3"]

    
strings = np.array(gene_expressed)
#adata.obs['genes_expressed'] = strings
coordinates = []
cx = adata.X
cx = cx.tocoo()
for i,j,v in zip(cx.row, cx.col, cx.data):
    coordinates += [[i,j,v]]
for coord in coordinates:
    coord[1] = adata.var_names[coord[1]]

#print(coordinates)

In [11]:
rowGeneExpression = {}

for coordinate in coordinates:
    row = coordinate[0]
    gene = coordinate[1]
    pvalue = coordinate[2]
    if not row in rowGeneExpression:
        rowGeneExpression[row] = 0
    rowGeneExpression[row] += pvalue

In [12]:
df = adata.obs
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()
dependent_variables =  [1 for item in numerical_values]

In [13]:
import torch.optim as optim


criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [17]:
from torch import tensor

t_dep = tensor(dependent_variables) # pertrubations


In [18]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[ 3.0170e+03,  8.7700e+02, -2.7640e+01,  ...,  9.0000e+00,
          1.7040e+03,  7.1100e+02],
        [ 2.4810e+03,  7.1300e+02, -2.7494e+01,  ...,  9.0000e+00,
          1.6140e+03,  6.6200e+02],
        [ 7.0300e+02,  3.3700e+02, -1.0468e+01,  ...,  3.0000e+00,
          9.0800e+02,  3.3700e+02],
        ...,
        [ 1.5220e+03,  5.2300e+02,  2.5142e+01,  ...,  6.0000e+00,
          1.4220e+03,  5.2300e+02],
        [ 1.1430e+03,  5.0300e+02,  1.4360e+01,  ...,  6.0000e+00,
          1.1850e+03,  5.0300e+02],
        [ 1.0310e+03,  4.2100e+02,  1.4572e+01,  ...,  5.0000e+00,
          1.1440e+03,  4.1900e+02]])

In [28]:
torch.manual_seed(442)


n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([-0.4629,  0.1386,  0.2409, -0.2262, -0.2632, -0.3147,  0.4876])

In [29]:
preds = (t_indep*coeffs).sum(axis=1)
loss = torch.abs(preds-t_dep).mean()
loss = calc_loss(coeffs, t_indep, t_dep)



loss = torch.abs(preds-t_dep).mean()
loss

tensor(1.0169)

In [30]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [31]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [32]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [33]:
t_indep*coeffs

tensor([[-0.1182,  0.0441, -0.1516, -0.0788, -0.2369, -0.1780,  0.2445],
        [-0.0972,  0.0358, -0.1508, -0.1522, -0.2369, -0.1686,  0.2276],
        [-0.0275,  0.0169, -0.0574,  0.0315, -0.0790, -0.0949,  0.1159],
        [-0.1340,  0.0427, -0.1337, -0.1075, -0.2369, -0.1816,  0.2245],
        [-0.1237,  0.0558,  0.1533, -0.1271, -0.1053, -0.1940,  0.3191],
        [-0.0732,  0.0319, -0.0026,  0.1336, -0.1316, -0.1593,  0.2180],
        [-0.0447,  0.0219, -0.0872, -0.1057, -0.2369, -0.1209,  0.1499],
        ...,
        [-0.0249,  0.0213, -0.0365,  0.0292, -0.0790, -0.0922,  0.1455],
        [-0.0525,  0.0241,  0.0423, -0.1754, -0.2106, -0.1383,  0.1651],
        [-0.0405,  0.0235,  0.1002, -0.0056, -0.1579, -0.1179,  0.1609],
        [-0.0829,  0.0412, -0.0634, -0.0135, -0.1053, -0.1744,  0.2748],
        [-0.0596,  0.0263,  0.1379, -0.0347, -0.1579, -0.1486,  0.1798],
        [-0.0448,  0.0253,  0.0788, -0.0577, -0.1579, -0.1238,  0.1730],
        [-0.0404,  0.0212,  0.0799,  0

In [34]:
t_indep = t_indep / vals

In [35]:
coeffs.requires_grad_()


tensor([-0.4629,  0.1386,  0.2409, -0.2262, -0.2632, -0.3147,  0.4876], requires_grad=True)

In [36]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [37]:
coeffs.grad

In [38]:
dependent_variables

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [39]:
independent_variables = pd.DataFrame(numerical_values)

In [40]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [41]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
len(trn_indep),len(val_indep)

(19739, 4934)

In [42]:
def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [43]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [44]:
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()

In [45]:
def train_model(epochs=30, lr=0.01):
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [46]:
coeffs = train_model(18, lr=0.2)

775.980; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 775.988; 953647.125; 

In [47]:
indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT']    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

{'nCount_RNA': tensor(-0.2832),
 'nFeature_RNA': tensor(0.1564),
 'tsne1': tensor(-0.3733),
 'tsne2': tensor(-0.1479),
 'cluster': tensor(-0.4332),
 'nCount_SCT': tensor(-0.2958),
 'nFeature_SCT': tensor(-0.0607)}

In [48]:
numerical_values

[[3017.0, 877.0, -27.6403728343912, 14.9666293340935, 9.0, 1704.0, 711.0],
 [2481.0, 713.0, -27.4936458839978, 28.9248851440496, 9.0, 1614.0, 662.0],
 [703.0, 337.0, -10.4681940980147, -5.98438948042782, 3.0, 908.0, 337.0],
 [3420.0, 850.0, -24.3679974111624, 20.4292847674974, 9.0, 1738.0, 653.0],
 [3158.0, 1111.0, 27.9521697403372, 24.1597381598606, 4.0, 1857.0, 928.0],
 [1869.0, 635.0, -0.470236382288022, -25.3987091482051, 5.0, 1525.0, 634.0],
 [1142.0, 436.0, -15.9061825808314, 20.0853153562297, 9.0, 1157.0, 436.0],
 [817.0, 427.0, -6.74039407642886, -23.9329848720015, 5.0, 961.0, 427.0],
 [1582.0, 558.0, -17.682802916336, 21.0055756282042, 9.0, 1397.0, 557.0],
 [728.0, 474.0, 26.1868881716573, -32.4273132631465, 7.0, 912.0, 473.0],
 [796.0, 386.0, 9.92445991071574, -16.4748745986804, 10.0, 954.0, 386.0],
 [2747.0, 980.0, 1.22976850612038, -10.5304907511738, 3.0, 1807.0, 930.0],
 [1341.0, 581.0, -11.6355124772875, -18.9496169441289, 5.0, 1305.0, 581.0],
 [753.0, 294.0, -10.70938422

In [49]:

acc(coeffs) + 1

##
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# pass in a h5ad file and get back 
#Predictions:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0]
# Perturbations:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0]

tensor(1.)

In [50]:
def benchmarkResults():
    pass

In [71]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)






adata_t = adata[
    ~(
        (adata.obs["cell_type"] == "CD4 T cells")
        & (adata.obs["condition"] == "stimulated")
    )
].copy()

cd4t_stim = adata[
    (
        (adata.obs["cell_type"] == "CD4 T cells")
        & (adata.obs["condition"] == "stimulated")
    )
].copy()








scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [72]:
model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [73]:
model.train(
    max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1: 100%|██████████████| 1/1 [00:13<00:00, 13.77s/it, v_num=1, train_loss_step=4.67e+3, train_loss_epoch=2.61e+4]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████████| 1/1 [00:13<00:00, 13.78s/it, v_num=1, train_loss_step=4.67e+3, train_loss_epoch=2.61e+4]


In [74]:
adata_t.obsm["scgen"] = model.get_latent_representation()

In [75]:
sc.pp.neighbors(adata_t, use_rep="scgen")
#sc.tl.umap(adata)

In [76]:
adata_t

AnnData object with n_obs × n_vars = 18995 × 15706
    obs: 'nCount_RNA', 'nFeature_RNA', 'tsne1', 'tsne2', 'condition', 'cluster', 'cell_type', 'replicate', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.4', 'seurat_clusters', '_scvi_batch', '_scvi_labels'
    var: 'name'
    uns: '_scvi_uuid', '_scvi_manager_uuid', 'neighbors'
    obsm: 'X_pca', 'X_umap', 'scgen'
    obsp: 'distances', 'connectivities'

In [77]:
pred, delta = model.predict(
    ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
)

# we annotate the predicted cells to distinguish them later from ground truth cells.
pred.obs["condition"] = "prediction"
print(pred)

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
AnnData object with n_obs × n_vars = 5560 × 15706
    obs: 'nCount_RNA', 'nFeature_RNA', 'tsne1', 'tsne2', 'condition', 'cluster', 'cell_type', 'replicate', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.4', 'seurat_clusters', '_scvi_batch', '_scvi_labels'
    var: 'name'
    obsm: 'X_pca', 'X_umap

In [78]:
pred.obs

,nCount_RNA,nFeature_RNA,tsne1,tsne2,condition,cluster,cell_type,replicate,nCount_SCT,nFeature_SCT,integrated_snn_res.0.4,seurat_clusters,_scvi_batch,_scvi_labels
index,,,,,,,,,,,,,,
AAACATACCATGCA-1,703.0,337,-10.468194,-5.984389,prediction,3,CD4 T cells,patient_1488,908.0,337,6,6,0,0
AAACATACGATGAA-1,1869.0,635,-0.470236,-25.398709,prediction,5,CD4 T cells,patient_1488,1525.0,634,2,2,0,0
AAACATACGCTTCC-1,817.0,427,-6.740394,-23.932985,prediction,5,CD4 T cells,patient_1256,961.0,427,0,0,0,0
AAACATACGTTGTG-1,796.0,386,9.924460,-16.474875,prediction,10,CD4 T cells,patient_1016,954.0,386,2,2,0,0
AAACATACTGCGTA-1,2747.0,980,1.229769,-10.530491,prediction,3,CD4 T cells,patient_107,1807.0,930,6,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCTACTGGTCTA-1,1208.0,550,-1.239073,-23.125451,prediction,5,CD4 T cells,patient_1015,1208.0,550,2,2,0,0
TTTGACTGAAGCAA-1,1253.0,531,-10.510576,-22.537674,prediction,5,CD4 T cells,patient_1488,1248.0,531,0,0,0,0
TTTGACTGCCGCTT-1,1810.0,587,-3.972385,-32.191004,prediction,5,CD4 T cells,patient_1244,1508.0,587,0,0,0,0


In [ ]:

train = adata
ctrl_adata = adata[
    ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
]
stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# concatenate pred, control and real CD4 T cells in to one object
eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [ ]:
sc.tl.pca(eval_adata)
sc.pl.pca(eval_adata, color="condition", frameon=False)

In [ ]:
cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [ ]:
sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
diff_genes

In [ ]:
array(['ISG15', 'IFI6', 'ISG20', ..., 'EEF1A1', 'FTH1', 'RGCC'],
      dtype=object)

In [ ]:
r2_value = model.reg_mean_plot(
    eval_adata,
    axis_keys={"x": "predicted stimulated", "y": "stimulated"},
    gene_list=diff_genes[:10],
    top_100_genes=diff_genes,
    labels={"x": "predicted", "y": "ground truth"},
    show=True,
    legend=False,
)

In [ ]:
sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
import pertpy as pt
import muon as mu
import scanpy as sc

In [ ]:
mdata = pt.dt.papalexi_2021()
for col in mdata.obs: print(col)

In [ ]:
sc.pp.normalize_total(mdata["rna"])
sc.pp.log1p(mdata["rna"])
sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
mu.prot.pp.clr(mdata["adt"])

In [ ]:
sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
sc.tl.umap(mdata["rna"])

In [ ]:
sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
print(adata_pert)

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)